In [24]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds

from tensorflow.keras import layers
import random
import shutil
import tensorflow_datasets as tfds

/Users/white/opt/anaconda3/envs/Transfer_Learning/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# let's set aside 20% of our Fibrosis images for validation
root_path = './dicom/NIH_1stPA_Norm_Fib'  ##Break so we don't accidentally run it again
fibrosis_images = os.listdir(root_path+'/Fibrosis') 
print(len(fibrosis_images))
num_for_validation = int(len(fibrosis_images)*.2)
validation_fibrosis_images = random.sample(fibrosis_images,num_for_validation)
len(validation_fibrosis_images)
for image in validation_fibrosis_images:
  source = f'{root_path}/Fibrosis/{image}'
  dest = f'{root_path}/Validation/{image}'
  shutil.move(source,dest)



637


In [22]:
#we will also copy an equal number of normals for validation
root_path = './dicom/NIH_1stPA_Norm_Fib'  ##Break so we don't accidentally run it again
normal_images = os.listdir(root_path+'/No Finding') 
normal_images
print(len(normal_images))
validation_normal_images = random.sample(normal_images,num_for_validation)
len(validation_normal_images)
for image in validation_normal_images:
  source = f'{root_path}/No Finding/{image}'
  dest = f'{root_path}/Validation/No Finding/{image}'
  shutil.move(source,dest)

6370


At this point we have 127 CXRs of no findings and 127 CXR with fibrosis in our  validation folder.
But we have ~500 Fibrosis and ~6k No Finding CXRs in our training folder.

In [37]:
# Let's us create a simple oversampled folder, increase our firbosis images by 10x
root_path = './dicom/NIH_1stPA_Norm_Fib/Training/Fibrosis Oversampled'
image_paths = os.listdir(root_path)
for filename in image_paths:
  splitF = filename.split('.') 
  for i in range(1,10):
    new_filename = f'{splitF[0]}-{i}.{splitF[1]}'
    source = f'{root_path}/{filename}'
    dest = f'{root_path}/{new_filename}'
    shutil.copy(source,dest)



In [ ]:
# Data augmentation